# Анализ баз данных книг (SQL)

# Содержание

1. Цель работы<br>
2. Подключение к базе данных<br>
3. Исследование данных<br>
4. Анализ БД<br>
    4.1. Расчет количества книг вышедших после 1 января 2000 года<br>
    4.2. Расчет количества обзоров и средней оценки для каждой книги<br>
    4.3. Поиск издательства, которое выпустило наибольшее число книг толще 50 страниц<br>
    4.4. Поиск автора с самой высокой средней оценкой книг (в учет берутся только книги с 50 и более оценками)<br>
    4.5. Расчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок<br>
5. Выводы

## Цель работы 

Анализ базы данных сервиса чтения книг на основе предоставленных данных

**Описание данных**

Таблица books - данные о книгах:
* book_id — идентификатор книги
* author_id — идентификатор автора
* title — название книги
* num_pages — количество страниц
* publication_date — дата публикации книги
* publisher_id — идентификатор издателя

Таблица authors - данные об авторах:
* author_id — идентификатор автора
* author — имя автора

Таблица publishers - данные об издательствах:
* publisher_id — идентификатор издательства
* publisher — название издательства

Таблица ratings - данные о пользовательских оценках книг
* rating_id — идентификатор оценки
* book_id — идентификатор книги
* username — имя пользователя, оставившего оценку
* rating — оценка книги

Таблица reviews - данные о пользовательских обзорах:
* review_id — идентификатор обзора
* book_id — идентификатор книги
* username — имя автора обзора
* text — текст обзора

## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

## Исследование данных

In [2]:
# Коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Запрос в БД
query = ''' 
SELECT *
FROM books;
'''

# Переменная хранящая результат запроса
books = pd.io.sql.read_sql(query, con=engine)

# Вывод результата
books.info()
books.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# Запрос в БД
query = ''' 
SELECT *
FROM authors;
'''

# Переменная хранящая результат запроса
authors = pd.io.sql.read_sql(query, con=engine)

# Вывод результата
authors.info()
authors.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Запрос в БД
query = ''' 
SELECT *
FROM publishers;
'''

# Переменная хранящая результат запроса
publishers = pd.io.sql.read_sql(query, con=engine)

# Вывод результата
publishers.info()
publishers.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# Запрос в БД
query = ''' 
SELECT *
FROM ratings;
'''

# Переменная хранящая результат запроса
ratings = pd.io.sql.read_sql(query, con=engine)

# Вывод результата
ratings.info()
ratings.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# Запрос в БД
query = ''' 
SELECT *
FROM reviews;
'''

# Переменная хранящая результат запроса
reviews = pd.io.sql.read_sql(query, con=engine)

# Вывод результата
reviews.info()
reviews.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Анализ БД
### Расчет количества книг вышедших после 1 января 2000 года

In [7]:
# Запрос в БД
query = ''' 
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';
'''

# Переменная хранящая результат запроса
fitrst_exc = pd.io.sql.read_sql(query, con=engine)

# Вывод результата
fitrst_exc

,count
0,819


Всего получилось 819 книг, которые публиковались после 1 января 2000 года

### Расчет количества обзоров и средней оценки для каждой книги

In [8]:
# Запрос в БД
query = ''' 
SELECT --b.book_id,
       b.title,
       COUNT(DISTINCT rev.review_id) AS reviews,
       AVG(rat.rating) AS avg_rating
FROM books AS b
LEFT OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
LEFT OUTER JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY b.book_id;
'''

# Переменная хранящая результат запроса
second_exc = pd.io.sql.read_sql(query, con=engine)
# Вывод результата
second_exc

,title,reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


### Поиск издательства, которое выпустило наибольшее число книг толще 50 страниц 

In [9]:
# Запрос в БД
query = ''' 
SELECT pub.publisher,
       COUNT(book_id) AS books
FROM books AS b
LEFT OUTER JOIN publishers AS pub ON b.publisher_id = pub.publisher_id
WHERE b.num_pages > 50
GROUP BY pub.publisher
ORDER BY books DESC
LIMIT 1;
'''

# Переменная хранящая результат запроса
third_exc = pd.io.sql.read_sql(query, con=engine)
# Вывод результата
third_exc

,publisher,books
0,Penguin Books,42


Определили издателя наибольшого числа книг 

### Поиск автора с самой высокой средней оценкой книг (в учет берутся только книги с 50 и более оценками)

In [10]:
# Запрос в БД
query = ''' 
SELECT a.author,
       AVG(r.rating) AS avg_rating
FROM books AS b
LEFT OUTER JOIN authors AS a ON b.author_id = a.author_id
LEFT OUTER JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY author
HAVING COUNT(r.rating) > 50
ORDER BY avg_rating DESC
LIMIT 1;
'''

# Переменная хранящая результат запроса
fourth_exc = pd.io.sql.read_sql(query, con=engine)
# Вывод результата
fourth_exc

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462


Определили автора с самой высокой оценкой

### Расчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
# Запрос в БД
query = ''' 
SELECT AVG(rev.reviews_count) AS avg_reviews
FROM (SELECT username,
             COUNT(review_id) AS reviews_count
      FROM reviews 
      GROUP BY username) AS rev
WHERE rev.username IN (SELECT username
                       FROM ratings
                       GROUP BY username
                       HAVING COUNT(rating) > 50);
'''
# Переменная хранящая результат запроса
fifth_exc = pd.io.sql.read_sql(query, con=engine)
# Вывод результата
fifth_exc

,avg_reviews
0,24.333333


In [12]:
#CODE REVIEW
# Запрос в БД
query = ''' 
SELECT username,
       count(*) as review
FROM reviews
GROUP BY username
ORDER BY review desc
LIMIT 5;
'''

pd.read_sql(query, con=engine)

,username,review
0,susan85,29
1,sfitzgerald,28
2,martinadam,27
3,eallen,26
4,lesliegibbs,26


## Выводы

* В ходе работы расчитано количество книг, опубликованных с начала 2000х - 819 книг
* Для каждой книги рассчитаны количество обзоров и средняя оценка
* Найдено издательство, выпустившее наибольшее число книг - Penguin Books с 42 книгами
* Найден самый рейтинговый автор - J.K. Rowling/Mary GrandPré со средним рейтингом 4.288462
* Рассчитано среднее число обзоров от самых ситающих пользователей